In [0]:
%sql

CREATE DATABASE IF NOT EXISTS gold;

In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_time;

CREATE TABLE gold.dim_time (
    id_time INT,
    nm_time STRING,
    nr_hour INT,
    nr_minutes INT,
    nm_time_day STRING
)
USING delta;

In [0]:
%sql
TRUNCATE TABLE gold.dim_time;

INSERT INTO gold.dim_time
WITH 
START_TIME AS (
    SELECT
        COALESCE(CAST(REPLACE(SUBSTRING(ended_at, 12, 5), ":", "") AS INT), -1) AS id_time,  -- Tratamento para id_time nulo
        CONCAT(
            LPAD(CAST(SUBSTRING(started_at, 12, 2) AS INT), 2, '0'),
            ':',
            LPAD(CAST(SUBSTRING(started_at, 15, 2) AS INT), 2, '0')
        ) AS nm_time,
        CAST(SUBSTRING(started_at, 12, 2) AS INT) AS nr_hour,
        CAST(SUBSTRING(started_at, 15, 2) AS INT) AS nr_minutes,
        CASE
            WHEN CAST(SUBSTRING(started_at, 12, 2) AS INT) >= 0 AND CAST(SUBSTRING(started_at, 12, 2) AS INT) < 6 THEN 'dawn'
            WHEN CAST(SUBSTRING(started_at, 12, 2) AS INT) >= 6 AND CAST(SUBSTRING(started_at, 12, 2) AS INT) < 12 THEN 'morning'
            WHEN CAST(SUBSTRING(started_at, 12, 2) AS INT) >= 12 AND CAST(SUBSTRING(started_at, 12, 2) AS INT) < 18 THEN 'afternoon'
            WHEN CAST(SUBSTRING(started_at, 12, 2) AS INT) >= 18 AND CAST(SUBSTRING(started_at, 12, 2) AS INT) < 24 THEN 'evening'
            ELSE 'unknown'
        END AS nm_time_day
    FROM hive_metastore.silver.trips
),
END_TIME AS (
    SELECT
        COALESCE(CAST(REPLACE(SUBSTRING(ended_at, 12, 5), ":", "") AS INT), -1) AS id_time,  -- Tratamento para id_time nulo
        CONCAT(
            LPAD(CAST(SUBSTRING(ended_at, 12, 2) AS INT), 2, '0'),
            ':',
            LPAD(CAST(SUBSTRING(ended_at, 15, 2) AS INT), 2, '0')
        ) AS nm_time,
        CAST(SUBSTRING(ended_at, 12, 2) AS INT) AS nr_hour,
        CAST(SUBSTRING(ended_at, 15, 2) AS INT) AS nr_minutes,
        CASE
            WHEN CAST(SUBSTRING(ended_at, 12, 2) AS INT) >= 0 AND CAST(SUBSTRING(ended_at, 12, 2) AS INT) < 6 THEN 'dawn'
            WHEN CAST(SUBSTRING(ended_at, 12, 2) AS INT) >= 6 AND CAST(SUBSTRING(ended_at, 12, 2) AS INT) < 12 THEN 'morning'
            WHEN CAST(SUBSTRING(ended_at, 12, 2) AS INT) >= 12 AND CAST(SUBSTRING(ended_at, 12, 2) AS INT) < 18 THEN 'afternoon'
            WHEN CAST(SUBSTRING(ended_at, 12, 2) AS INT) >= 18 AND CAST(SUBSTRING(ended_at, 12, 2) AS INT) < 24 THEN 'evening'
            ELSE 'unknown'
        END AS nm_time_day
    FROM hive_metastore.silver.trips
),
UNION_TABLES AS (
    SELECT * FROM START_TIME
    UNION
    SELECT * FROM END_TIME
)
SELECT id_time, nm_time, nr_hour, nr_minutes, nm_time_day
FROM UNION_TABLES
ORDER BY id_time;

num_affected_rows,num_inserted_rows
175720,175720


In [0]:
%sql

SELECT * FROM gold.dim_time limit 10

id_time,nr_time,nr_hour,nr_minutes,nm_time_day
0,00:00,0,0,dawn
1,00:01,0,1,dawn
2,00:02,0,2,dawn
3,00:03,0,3,dawn
4,00:04,0,4,dawn
5,00:05,0,5,dawn
6,00:06,0,6,dawn
7,00:07,0,7,dawn
8,00:08,0,8,dawn
9,00:09,0,9,dawn


In [0]:
%sql

DROP TABLE IF EXISTS gold.dim_date;

CREATE TABLE gold.dim_date (
    id_date INT,
    dt_date DATE,
    nr_day INT,
    nr_month INT,
    nr_year INT,
    nr_quarter INT,
    nm_day_week STRING,
    nr_day_year INT
)
USING delta;

In [0]:
%sql

TRUNCATE TABLE gold.dim_date;

INSERT INTO gold.dim_date
WITH STAGE AS
(
    SELECT DISTINCT CAST(birthday AS Date) AS dt_date FROM silver.riders
    UNION
    SELECT DISTINCT CAST("Date" AS Date) AS dt_date FROM silver.payment
    UNION 
    SELECT DISTINCT CAST(account_start_date AS Date) AS dt_date FROM silver.riders
    UNION 
    SELECT DISTINCT CAST(account_end_date AS Date) AS dt_date FROM silver.riders
    UNION 
    SELECT DISTINCT CAST(started_At AS Date) AS dt_date FROM silver.trips
    UNION 
    SELECT DISTINCT CAST(ended_At AS Date) AS dt_date FROM silver.trips
)

SELECT  COALESCE(CAST(DATE_FORMAT(dt_date, 'yyyyMMdd') AS INT), -1) id_date,
        dt_date,
        DAY(dt_date) AS nr_day,
        MONTH(dt_date) AS nr_month,
        YEAR(dt_date) AS nr_year,
        QUARTER(dt_date) AS nr_quarter,
        CASE DAYOFWEEK(dt_date)
            WHEN 1 THEN 'Sunday'
            WHEN 2 THEN 'Monday'
            WHEN 3 THEN 'Tuesday'
            WHEN 4 THEN 'Wednesday'
            WHEN 5 THEN 'Thursday'
            WHEN 6 THEN 'Friday'
            WHEN 7 THEN 'Saturday'
            ELSE 'Unknown'
        END AS nm_day_week,
        DAYOFYEAR(dt_date) AS nr_day_year
FROM STAGE
ORDER BY 1

num_affected_rows,num_inserted_rows
19319,19319


In [0]:
%sql

SELECT *
FROM gold.dim_date
ORDER BY 1 DESC
LIMIT 10

id_date,dt_date,nr_day,nr_month,nr_year,nr_quarter,nm_day_week,nr_day_year
20220212,2022-02-12,12,2,2022,1,Saturday,43
20220211,2022-02-11,11,2,2022,1,Friday,42
20220210,2022-02-10,10,2,2022,1,Thursday,41
20220209,2022-02-09,9,2,2022,1,Wednesday,40
20220208,2022-02-08,8,2,2022,1,Tuesday,39
20220207,2022-02-07,7,2,2022,1,Monday,38
20220206,2022-02-06,6,2,2022,1,Sunday,37
20220205,2022-02-05,5,2,2022,1,Saturday,36
20220204,2022-02-04,4,2,2022,1,Friday,35
20220203,2022-02-03,3,2,2022,1,Thursday,34


In [0]:
%sql

DROP TABLE IF EXISTS gold.dim_station;

CREATE TABLE gold.dim_station (
    id_station STRING,
    nm_station STRING,
    nr_latitude DECIMAL(9,6),
    nr_longitude DECIMAL(9,6)
)
USING delta;

In [0]:
%sql
TRUNCATE TABLE gold.dim_station;

INSERT INTO gold.dim_station
SELECT
    Station_ID as id_station,
    Name as nm_station,
    CAST(Latitude AS DECIMAL(9,6)) as nr_latitude,
    CAST(Longitude AS DECIMAL(9,6)) as nr_longitude
FROM
    silver.stations;

num_affected_rows,num_inserted_rows
838,838


In [0]:
%sql

SELECT *
FROM gold.dim_station
LIMIT 10

id_station,nm_station,nr_latitude,nr_longitude
525,Glenwood Ave & Touhy Ave,42.012701,-87.666058
KA1503000012,Clark St & Lake St,41.885795,-87.631101
637,Wood St & Chicago Ave,41.895634,-87.672069
13216,State St & 33rd St,41.834734,-87.625828
18003,Fairbanks St & Superior St,41.895808,-87.620253
KP1705001026,LaSalle Dr & Huron St,41.894877,-87.632326
13253,Lincoln Ave & Waveland Ave,41.948797,-87.675278
KA1503000044,Rush St & Hubbard St,41.890173,-87.626185
KA1504000140,Winchester Ave & Elston Ave,41.924037,-87.676415
TA1305000032,Clinton St & Madison St,41.882242,-87.641066


In [0]:
%sql

DROP TABLE IF EXISTS gold.dim_rider;

CREATE TABLE gold.dim_rider (
    id_rider INT,
    nm_first_name STRING,
    nm_last_name STRING,
    dt_birthday DATE,
    bl_is_member BOOLEAN,
    dt_account_start DATE,
    dt_account_end DATE
)
USING delta;

In [0]:
%sql

TRUNCATE TABLE gold.dim_rider;

INSERT INTO gold.dim_rider
SELECT
    Rider_Id AS id_rider,
    "First" AS nm_first_name,
    "Last" AS nm_last_name,
    CAST(Birthday AS DATE) AS dt_birthday,
    Is_member AS bl_is_member,
    CAST(Account_start_date AS DATE) AS dt_account_start,
    CAST(Account_end_date AS DATE) AS dt_account_end
FROM silver.riders

num_affected_rows,num_inserted_rows
75000,75000


In [0]:
%sql

SELECT *
FROM gold.dim_rider
LIMIT 10

id_rider,nm_first_name,nm_last_name,dt_birthday,bl_is_member,dt_account_start,dt_account_end
1000,First,Last,1989-02-13,true,2019-04-23,null
1001,First,Last,1976-08-10,true,2019-11-01,2020-09-01
1002,First,Last,1998-08-10,true,2022-02-04,null
1003,First,Last,1999-03-29,false,2019-08-26,null
1004,First,Last,1969-04-11,true,2019-09-14,null
1005,First,Last,1974-08-27,false,2020-03-24,null
1006,First,Last,2004-01-30,true,2020-11-27,2021-12-01
1007,First,Last,1988-01-11,false,2016-12-11,null
1008,First,Last,1987-02-21,true,2021-03-28,2021-07-01
1009,First,Last,1981-02-07,true,2020-06-12,2021-11-01


In [0]:
%sql

DROP TABLE IF EXISTS gold.fact_payment;

CREATE TABLE gold.fact_payment (
    id_date INT,
    id_rider INT,
    vl_account_amount DECIMAL(9,2),
    nr_rider_age INTEGER
)
USING delta;

In [0]:
%sql

TRUNCATE TABLE gold.fact_payment;

INSERT INTO gold.fact_payment
SELECT 
    CASE 
        WHEN pay.`date` IS NOT NULL THEN CAST(CONCAT(SUBSTRING(pay.`date`, 1, 4), SUBSTRING(pay.`date`, 6, 2), SUBSTRING(pay.`date`, 9, 2)) AS INT)
        ELSE -1
    END AS id_date,
    pay.rider_id as id_rider,
    pay.amount as vl_account_amount,
    DATEDIFF(YEAR, TO_DATE(rid.dt_birthday), TO_DATE(pay.`date`))  as nr_rider_age
FROM silver.payment pay
JOIN gold.dim_rider rid
    ON pay.rider_id = rid.id_rider

num_affected_rows,num_inserted_rows
1946607,1946607


In [0]:
%sql

SELECT *
FROM gold.fact_payment
LIMIT 10

id_date,id_rider,vl_account_amount,nr_rider_age
20190501,1000,9.00,30
20190601,1000,9.00,30
20190701,1000,9.00,30
20190801,1000,9.00,30
20190901,1000,9.00,30
20191001,1000,9.00,30
20191101,1000,9.00,30
20191201,1000,9.00,30
20200101,1000,9.00,30
20200201,1000,9.00,30


In [0]:
%sql

DROP TABLE IF EXISTS gold.fact_trip;

CREATE TABLE gold.fact_trip (
    id_date_start INT,
    id_time_start INT,
    id_station_start STRING,
    id_date_end INT,
    id_time_end INT,
    id_station_end STRING,
    id_rider INT,
    nr_rider_age INT,
    nr_trip_minutes_duration INT
)
USING delta;

In [0]:
%sql

TRUNCATE TABLE gold.fact_trip;

INSERT INTO gold.fact_trip
SELECT
    COALESCE(CAST(DATE_FORMAT(started_at, 'yyyyMMdd') AS INT), -1) AS id_date_start,
    COALESCE(CAST(REPLACE(SUBSTRING(started_at, 12, 5), ":", "") AS INT), -1) AS id_time_start,
    tri.start_station_id AS id_station_start,
    COALESCE(CAST(DATE_FORMAT(ended_at, 'yyyyMMdd') AS INT), -1) AS id_date_end,
    COALESCE(CAST(REPLACE(SUBSTRING(ended_at, 12, 5), ":", "") AS INT), -1) AS id_time_end,
    tri.end_station_id AS id_station_end,
    rider_id AS id_rider,
    DATEDIFF(YEAR, rid.dt_birthday, CAST(tri.Started_At AS DATE)) AS Rider_Age,
    DATEDIFF(MINUTE, CAST(tri.Started_At AS TIMESTAMP), CAST(tri.Ended_At AS TIMESTAMP)) AS nr_trip_minutes_duration
FROM
    silver.trips AS tri
    JOIN gold.dim_rider AS rid
        ON rid.id_rider = tri.rider_id;

num_affected_rows,num_inserted_rows
4584921,4584921


In [0]:
%sql

select *
FROM gold.fact_trip
limit 10

id_date_start,id_time_start,id_station_start,id_date_end,id_time_end,id_station_end,id_rider,nr_rider_age,nr_trip_minutes_duration
20210212,1614,525,20210212,1621,660,71934,37,6
20210214,1752,525,20210214,1812,16806,47854,38,19
20210209,1910,KA1503000012,20210209,1919,TA1305000029,70870,33,8
20210202,1749,637,20210202,1754,TA1305000034,58974,19,4
20210223,1507,13216,20210223,1522,TA1309000055,39608,71,15
20210224,1543,18003,20210224,1549,KP1705001026,36267,27,5
20210201,1747,KP1705001026,20210201,1748,KP1705001026,50104,33,0
20210211,1833,18003,20210211,1835,18003,19618,23,1
20210227,1513,KP1705001026,20210227,1536,KP1705001026,16732,19,22
20210220,859,KP1705001026,20210220,917,KP1705001026,57068,47,17
